In [8]:
# БИХ-фильтра низких частот (ФНЧ)| фильтр Баттерворта

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import fxpmath as fxp

In [9]:

# Параметры фильтра
cutoff_freq = 10000.0  # Частота среза 10 кГц
sample_rate = 44100    # Частота дискретизации (стандартная для аудио)
order = 4              # Порядок фильтра (выше порядок = круче срез)

# Рассчитываем нормированную частоту среза (относительно частоты Найквиста)
nyquist_freq = 0.5 * sample_rate
normalized_cutoff = cutoff_freq / nyquist_freq

# Создаем фильтр Баттерворта
b, a = signal.butter(order, normalized_cutoff, btype='low', analog=False)

In [12]:
'''
sum a[i]*y[n-i] = sum b[i]*x[n-i], for i in range(len(a))
'''

fxp_size = 16
fraction_part = 12

def to_fxp(x: float) -> str:
     return f'{'-' if x<0 else ''}{fxp_size + fraction_part}\'b{fxp.Fxp(abs(x), n_word=fxp_size+fraction_part, n_frac=fraction_part).bin()}'

assert a[0] == 1, 'Sowwy, i dont reawy to generate code with non one a[0] Uwu'


code = f'''
module preprocess_hipass(
\tinput clk,
\tinput rst,
\tinput i_valid,
\tinput  signed[FXP_SIZE-1: 0] i_sample,
\toutput signed[FXP_SIZE-1: 0] o_sample
);

\tlocalparam FXP_SIZE = {fxp_size};
\tlocalparam FXP_FRAC = {fraction_part};
\tlocalparam COMP_SIZE = FXP_SIZE + FXP_FRAC; // size of numbers in computations
\t
\t// for more precision
\twire signed[COMP_SIZE-1] wide_sample;
\tsigned_expand#(
\t     .operand_size  (FXP_SIZE),
\t     .expansion_size(FXP_FRAC) 
\t) ins_sample_expand (
\t     .in(i_sample),
\t     .out(wide_sample)
\t);
\t
\t
\twire[COMP_SIZE-1: 0] y_next;
\tlogic[COMP_SIZE-1:0] y[{order-1}];
\tlogic[COMP_SIZE-1:0] x[{order}];
\t
\tassign x[0] = wide_sample;
'''

code += f'\talways_ff@(posedge clk) begin\n'
for i in range(order-1):
     code += f'''
\t\tif(rst) y[{i}] <= \'0;
\t\telse y[{i}] <= {'y_next' if i == 0 else f'y[{i-1}]'};
'''
code += f'\tend\n\n'

code += f'\talways_ff@(posedge clk) begin\n'
for i in range(order):
     code += f'''
\t\tif(rst) x[{i}] <= \'0;
\t\telse x[{i}] <= x[{i-1}];
'''
code += f'\tend\n'

for i in range(order):
     if i!=0:
          code += f'\tlocalparam a_{i} = {to_fxp(a[i])};\n'
     code += f'\tlocalparam b_{i} = {to_fxp(b[i])};\n'

code += '\n'*3

code += '\tassign y_next = $signed('
code += ' + '.join(f'b_{i} * x[{i}]' for i in range(len(a))) + ' - ' + ' - '.join(f'a_{i} * y[{i-1}]' for i in range(1, len(a)))
code += ') >>> FXP_FRAC;\n'
code += 'endmodule\n'
print(code)



module preprocess_hipass(
	input clk,
	input rst,
	input i_valid,
	input  signed[FXP_SIZE-1: 0] i_sample,
	output signed[FXP_SIZE-1: 0] o_sample
);

	localparam FXP_SIZE = 16;
	localparam FXP_FRAC = 12;
	localparam COMP_SIZE = FXP_SIZE + FXP_FRAC; // size of numbers in computations
	
	// for more precision
	wire signed[COMP_SIZE-1] wide_sample;
	signed_expand#(
	     .operand_size  (FXP_SIZE),
	     .expansion_size(FXP_FRAC) 
	) ins_sample_expand (
	     .in(i_sample),
	     .out(wide_sample)
	);
	
	
	wire[COMP_SIZE-1: 0] y_next;
	logic[COMP_SIZE-1:0] y[3];
	logic[COMP_SIZE-1:0] x[4];
	
	assign x[0] = wide_sample;
	always_ff@(posedge clk) begin

		if(rst) y[0] <= '0;
		else y[0] <= y_next;

		if(rst) y[1] <= '0;
		else y[1] <= y[0];

		if(rst) y[2] <= '0;
		else y[2] <= y[1];
	end

	always_ff@(posedge clk) begin

		if(rst) x[0] <= '0;
		else x[0] <= x[-1];

		if(rst) x[1] <= '0;
		else x[1] <= x[0];

		if(rst) x[2] <= '0;
		else x[2] <= x[1];

		if(rst) x[3] <= '0;
		else x[3] <= x[2]